In [121]:
import pandas as pd
import numpy as np
from scipy import stats
from numpy import mean
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
df = pd.read_csv("./weatherAUS.csv")
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


First of all I imported the data and take a look at the first 5 row, so I have some idea what kind of chart it is. Now I sort the value of each column to see and spot the difference.

You can see below that the first 4 column have a very less value compared to other, which mean that they have many missing data so it would be better to remove them so we do not make any mistake
from using the missing data

In [2]:
df.count().sort_values()

Sunshine          75625
Evaporation       82670
Cloud3pm          86102
Cloud9am          89572
Pressure9am      130395
Pressure3pm      130432
WindDir9am       134894
WindGustDir      135134
WindGustSpeed    135197
Humidity3pm      140953
WindDir3pm       141232
Temp3pm          141851
RainTomorrow     142193
RainToday        142199
Rainfall         142199
WindSpeed3pm     142398
Humidity9am      142806
Temp9am          143693
WindSpeed9am     143693
MinTemp          143975
MaxTemp          144199
Location         145460
Date             145460
dtype: int64

In [3]:
remove = df.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am','Date'],axis=1)
remove.shape


(145460, 18)

Base on the excel sheet there are still many rows that contain an empty data so it woulde be best if I remove that too.
As the result, around 30k rows have been removed as they have an empty data.

In [4]:
remove_row = remove.dropna()
remove_row
remove_row.shape

(112925, 18)

In [5]:
remove_row.dtypes
# For this command I check for all the categorical of each column and I want to seperate them. As I do my research I found out that the algorithms trend to work better when the data is on the
# same scale as each other, that why I am seperating them and only scaling the one with float 64 type.

Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

In [6]:

num = [var for var in remove_row.columns if remove_row[var].dtype =='float64']

num


['MinTemp',
 'MaxTemp',
 'Rainfall',
 'WindGustSpeed',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Temp9am',
 'Temp3pm']

Here I got all the numerical columns out but they are not yet seperated from the table so I will be looping over them and apply the scaling.

In [7]:
for col in num:
    remove_row[col] = preprocessing.scale(remove_row[col])

remove_row.head()

C:\Users\uSeR\AppData\Local\Temp/ipykernel_19056/3916105241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remove_row[col] = preprocessing.scale(remove_row[col])
C:\Users\uSeR\AppData\Local\Temp/ipykernel_19056/3916105241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remove_row[col] = preprocessing.scale(remove_row[col])
C:\Users\uSeR\AppData\Local\Temp/ipykernel_19056/3916105241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Albury,0.117567,-0.108221,-0.206661,W,0.241214,W,WNW,0.577742,0.524408,0.190140,-1.380413,-1.382962,-1.142455,-0.088435,-0.047870,No,No
1,Albury,-0.841802,0.206845,-0.276405,WNW,0.241214,NNW,WSW,-1.339742,0.291310,-1.237561,-1.235963,-0.970598,-1.041848,-0.041228,0.317768,No,No
2,Albury,0.037620,0.292772,-0.276405,WSW,0.391345,W,WSW,0.457900,0.757507,-1.554828,-0.995214,-1.397181,-0.912497,0.556724,0.156887,No,No
3,Albury,-0.553991,0.622159,-0.276405,NE,-1.260094,SE,E,-0.500842,-1.223831,-1.184683,-1.669313,0.024764,-0.323229,0.100392,0.639531,No,No
4,Albury,0.773137,1.237969,-0.160165,W,0.016018,ENE,NW,-0.980214,0.058211,0.771796,-0.850764,-0.942159,-1.300551,0.053185,1.107548,No,No


Now this part I did some research on the internet and found out that by removing outliers I can prevent some error as a large outliers can affect the accuracy of linear regression models.
I then simpliy remove the row that have an outlier from the dataset by looking at the Z score. Anything that is beyond 3 standard deviations will be considered outliers
But first thing first I need to convert everything into a positive value.

In [8]:
bb = np.abs(remove_row._get_numeric_data())
print(bb)

         MinTemp   MaxTemp  Rainfall  WindGustSpeed  WindSpeed9am  \
0       0.117567  0.108221  0.206661       0.241214      0.577742   
1       0.841802  0.206845  0.276405       0.241214      1.339742   
2       0.037620  0.292772  0.276405       0.391345      0.457900   
3       0.553991  0.622159  0.276405       1.260094      0.500842   
4       0.773137  1.237969  0.160165       0.016018      0.980214   
...          ...       ...       ...            ...           ...   
145454  1.465392  0.265754  0.276405       0.734636      0.021471   
145455  1.577319  0.036615  0.276405       0.734636      0.261157   
145456  1.449403  0.235487  0.276405       1.410225      0.261157   
145457  1.161592  0.464626  0.276405       0.284243      0.740528   
145458  0.777844  0.478947  0.276405       0.959832      0.261157   

        WindSpeed3pm  Humidity9am  Humidity3pm  Pressure9am  Pressure3pm  \
0           0.524408     0.190140     1.380413     1.382962     1.142455   
1           0.29131

In [9]:
ff = remove_row[(bb<3).all(axis=1)]
ff.shape
# For the code above it will only return True if all the value within the row are lesser than 3
# If it is greater than 3 that rows is removed instantly.


(107868, 18)

Now that till this point I have deal with all the numerical data, scaling and removing all the outliers next step will be dealing with the categorical data.

In [10]:
#First I will convert all the Yes and No to be 0 and 1
ff['RainToday'].replace({'No': 0, 'Yes': 1},inplace = True)
ff['RainTomorrow'].replace({'No': 0, 'Yes': 1},inplace = True)
ff.head()

C:\Users\uSeR\anaconda3\envs\iccs161 ver2\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Albury,0.117567,-0.108221,-0.206661,W,0.241214,W,WNW,0.577742,0.524408,0.190140,-1.380413,-1.382962,-1.142455,-0.088435,-0.047870,0,0
1,Albury,-0.841802,0.206845,-0.276405,WNW,0.241214,NNW,WSW,-1.339742,0.291310,-1.237561,-1.235963,-0.970598,-1.041848,-0.041228,0.317768,0,0
2,Albury,0.037620,0.292772,-0.276405,WSW,0.391345,W,WSW,0.457900,0.757507,-1.554828,-0.995214,-1.397181,-0.912497,0.556724,0.156887,0,0
3,Albury,-0.553991,0.622159,-0.276405,NE,-1.260094,SE,E,-0.500842,-1.223831,-1.184683,-1.669313,0.024764,-0.323229,0.100392,0.639531,0,0
4,Albury,0.773137,1.237969,-0.160165,W,0.016018,ENE,NW,-0.980214,0.058211,0.771796,-0.850764,-0.942159,-1.300551,0.053185,1.107548,0,0


In [11]:
#So in order to deal with the Wind direction I found a method called "One Hot Encoding" which essentially find all the unique values and create a separate column for each of them.
#Now first of all we need to once again identify which of the column are categorical by creating a list just like how we did in the numerical list.
# Simply looping through all the column and add variable which are typed object. 

categ = [var for var in ff.columns if ff[var].dtype=='object']
print(categ)

['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']


In [12]:
for col in categ:
    print(np.unique(ff[col]))

['Adelaide' 'Albury' 'AliceSprings' 'BadgerysCreek' 'Ballarat' 'Bendigo'
 'Brisbane' 'Cairns' 'Canberra' 'Cobar' 'CoffsHarbour' 'Dartmoor' 'Darwin'
 'GoldCoast' 'Hobart' 'Katherine' 'Launceston' 'Melbourne'
 'MelbourneAirport' 'Mildura' 'Moree' 'MountGambier' 'Nhil' 'NorahHead'
 'NorfolkIsland' 'Nuriootpa' 'PearceRAAF' 'Perth' 'PerthAirport'
 'Portland' 'Richmond' 'Sale' 'Sydney' 'SydneyAirport' 'Townsville'
 'Tuggeranong' 'Uluru' 'WaggaWagga' 'Walpole' 'Watsonia' 'Williamtown'
 'Witchcliffe' 'Wollongong' 'Woomera']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']


In [13]:
# We successfully aquired all the unique value from the column now it's time to apply One Hot Coding by using buildin from pandas
df1 = pd.get_dummies(ff, columns = categ)
df1.head()

#Now you can see all the wind direction got their own unique column 

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,0.117567,-0.108221,-0.206661,0.241214,0.577742,0.524408,0.190140,-1.380413,-1.382962,-1.142455,...,0,0,0,0,0,0,0,0,1,0
1,-0.841802,0.206845,-0.276405,0.241214,-1.339742,0.291310,-1.237561,-1.235963,-0.970598,-1.041848,...,0,0,0,0,0,0,0,0,0,1
2,0.037620,0.292772,-0.276405,0.391345,0.457900,0.757507,-1.554828,-0.995214,-1.397181,-0.912497,...,0,0,0,0,0,0,0,0,0,1
3,-0.553991,0.622159,-0.276405,-1.260094,-0.500842,-1.223831,-1.184683,-1.669313,0.024764,-0.323229,...,0,0,0,0,0,0,0,0,0,0
4,0.773137,1.237969,-0.160165,0.016018,-0.980214,0.058211,0.771796,-0.850764,-0.942159,-1.300551,...,0,1,0,0,0,0,0,0,0,0


Now moving on to the next step where we will predicting if it will rain tomorrow.


In [23]:


X = df1.loc[:, df1.columns!='RainTomorrow'] #setting X (input)to go to all the variable and column that is not RainTomorrow. Note that some of the variable might not needed in order to predit
                                            # if it will rain tomorrow of not but luckily our model can figure it out and will assign a very small value to the variable making it have no impact
                                            # on the output

# Setting y(output) to RainTomorrow                                            
y = df1.RainTomorrow 


After assigning input and output we will seperate the data into 25% of testing and 75% of training. This step is called cross-validation, it helps to evaluate the model's performance.
Once we seperate them we are ready to implement the model and by that I imported LogiticRegression 

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

reglog = LogisticRegression(max_iter=3000).fit(X_train, y_train)
predict = reglog.predict(X_test)

Come to this part where I will be using KFold cross validation which is a resampling procedure that used to estimate the skill of the model on new data. This method is very accurate as it evaluate by using all the data to test our model.
I wil be spliting the data into 5 folds and calculate the accuracy using cross val score method and compare it to accuracy score method and find the average among them


In [126]:

# split data into 5 folds and shuffle 1 to make sure we get same shuffle everytime.
aa = KFold(n_splits=10, random_state=1, shuffle=True)
# using the cross val score to implement the cross validation by creatinga list
# This give out the percentage of correct predictions.
output = mean(cross_val_score(reglog, X, y, scoring='accuracy', cv=aa))
#printing out the mean of the 5 accuracy score we calculated above
print(output)

#Compare to using accuracy score
rr = accuracy_score(predict, y_test)
print(rr)

print("Average accuracy score:",(output+rr)/2)

0.8565561770232835
0.8564171023843957
Average accuracy score: 0.8564866397038395


The result was pretty good as it was 0.856 which can be written as 85.6% which is pretty high so now moving to the last part of this model which is to predict the out come of the rain with the accuray we calculated previously 


In [116]:
print(reglog.predict(X_test[0:100]))
# 0 mean not rain
# 1 mean rain

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1]


In [128]:
print(classification_report(predict,y_test))

print(f"""
accuracy_score: {(output+rr)/2}
roc_auc_score: {roc_auc_score(predict, y_test)}
""")

# After some research I was inspired to present the final data in this form

              precision    recall  f1-score   support

           0       0.95      0.88      0.91     23126
           1       0.50      0.73      0.59      3841

    accuracy                           0.86     26967
   macro avg       0.72      0.80      0.75     26967
weighted avg       0.89      0.86      0.87     26967


accuracy_score: 0.8564866397038395
roc_auc_score: 0.8026289280217002

